## Get and Upload Complete Data to MariaDB and AWS S3
From the official nintendo website the following fields were obtained:
- Image
- Release Date
- Description

In [1]:
import os
import json
import requests
from time import sleep

# Database
import pandas as pd
import psycopg2
import mysql.connector as mariadb

# AWS S3
import boto3
from io import BytesIO

## Upload images to S3

In [4]:
df = pd.read_csv('../dataset/switch-games-id.csv', sep=',')
df.head()

,title,game_url,image_url,system,publisher,developer,genre,num_players,release_date,review,review_conclusion,num_votes,rating,id
0,The DioField Chronicle,https://www.nintendolife.com/reviews/nintendo-...,https://images.nintendolife.com/d9a6ef84ec8cf/...,Nintendo Switch,Square Enix,Square Enix,RPG Simulation Strategy,1,22nd Sep 2022,Published by Square Enix and developed with as...,Despite its shortcomings The DioField Chronic...,0,0.0,0f0191a3
1,The Legend of Heroes: Trails from Zero,https://www.nintendolife.com/reviews/nintendo-...,https://images.nintendolife.com/004c2655b08cc/...,Nintendo Switch,NIS America,Nihon Falcom,RPG,1,27th Sep 2022,We dont blame you if you’re intimidated by Fal...,In a series renowned for its stellar writing ...,0,0.0,f2af5f5f
2,NBA 2K23,https://www.nintendolife.com/reviews/nintendo-...,https://images.nintendolife.com/60aff0dbdc402/...,Nintendo Switch,2K,Visual Concepts,Sports,4 (10 Online),9th Sep 2022,Its once again reached that time of year when ...,NBA 2K23 makes the inevitable downgrades and c...,2,0.0,a8996117
3,Temtem,https://www.nintendolife.com/reviews/nintendo-...,https://images.nintendolife.com/c6c585ce49b59/...,Nintendo Switch,Humble Games,Crema Games,Action Adventure RPG,1 (999 Online),6th Sep 2022,At the beginning of our Temtem adventure we h...,Even without the endgame content Temtem is wo...,5,7.8,e2f10932
4,Made in Abyss: Binary Star Falling into Darkness,https://www.nintendolife.com/reviews/nintendo-...,https://images.nintendolife.com/063177f5b67f4/...,Nintendo Switch,Spike Chunsoft US,Chime,Action RPG,1,2nd Sep 2022,Made in Abyss feels like an anime/manga ripe f...,Made in Abyss is an odd game to recommend; the...,2,0.0,51ff4f64


In [29]:
s3_client = boto3.client('s3')
bucket = 'nyzyss-the-eternal'

for id, image in zip(df['id'], df['image_url']):
    print(id, image)
    res = requests.get(image, stream = True)
    buffer = BytesIO(res.content)
    s3_client.upload_fileobj(
        buffer,
        bucket,
        id,
        ExtraArgs={"ContentType": "image/png"},
    )

0f0191a3 https://images.nintendolife.com/d9a6ef84ec8cf/250x183.jpg
f2af5f5f https://images.nintendolife.com/004c2655b08cc/250x183.jpg
a8996117 https://images.nintendolife.com/60aff0dbdc402/250x183.jpg
e2f10932 https://images.nintendolife.com/c6c585ce49b59/250x183.jpg
51ff4f64 https://images.nintendolife.com/063177f5b67f4/250x183.jpg
4377dbf1 https://images.nintendolife.com/80972d7551e36/250x183.jpg
751a7bbf https://images.nintendolife.com/ddac5a4cb6e84/250x183.jpg
dc506d3b https://images.nintendolife.com/400345afa3efd/250x183.jpg
438bc630 https://images.nintendolife.com/bcf32146c1a25/250x183.jpg
7796d27e https://images.nintendolife.com/59614e734f16d/250x183.jpg
4e293730 https://images.nintendolife.com/9a5d6072fa8c4/250x183.jpg
31409bf6 https://images.nintendolife.com/2a0ab97ceb646/250x183.jpg
b2f2b75f https://images.nintendolife.com/d42379b22a491/250x183.jpg
ff64c4ba https://images.nintendolife.com/5687e7852a9ba/250x183.jpg
a1af38c1 https://images.nintendolife.com/038c0c74d9369/250x183

## Upload data to DB

In [7]:
connection = psycopg2.connect(
    host="containers-us-west-63.railway.app",
    database="railway",
    user="postgres",
    password="********",
    port="8077"
)
cursor = connection.cursor()
for ind in range(df.shape[0]):
    game = df.iloc[ind]
    id = game['id']
    title = game['title']
    release_date = game['release_date']
    image = 'https://nyzyss-the-eternal.s3.amazonaws.com/' + id
    game_description = game['review'][:500]
    print(id)

    query = '''
        INSERT INTO switch_games VALUES ('{0}', '{1}', '{2}', '{3}', '{4}');
    '''.format(id, title, release_date, image, game_description)

    cursor.execute(query)

connection.commit()

0f0191a3
f2af5f5f
a8996117
e2f10932
51ff4f64
4377dbf1
751a7bbf
dc506d3b
438bc630
7796d27e
4e293730
31409bf6
b2f2b75f
ff64c4ba
a1af38c1
b29030ea
6ae91a77
54ce4908
889702e7
95d72d02
646338d6
d9b9c233
a2af77a5
08b82442
bf50d7d2
c3779274
07aa6c7c
ba5884fc
03e90c97
b54f53ed
cd2eaf42
473fa16b
150e4c2f
9f04c0a3
2d21ea6b
205c3514
8b92a357
6c44e10d
90eef2ce
582ef9ce
e991580e
3def811d
8e974e1f
69929c7f
adf96fa0
5d25498b
c2fa5086
4c6f5c8f
147d5275
b504a5e6
64bbe7e0
8853dffd
28207116
d3922b8c
437c3f28
2be54b4d
b9324ae7
51782cec
591b470a
9b669a81
16aa9090
4cfbba7c
85c8956f
7f75ffa2
70b435dd
cd118da3
cbb7c571
61809917
16c7bee5
711db752
328114bc
dba4ad4f
d9fc5767
19df991c
869d26d3
c1c36b5b
b3e7de53
05d27b82
3ae187ae
a7270539
7d911774
b307500d
8b22873c
314f68d8
675ca3f5
51767e63
01d940f1
5174a91d
fbc872d6
ffa46d12
a8ec626b
b356df6e
3a90b8f5
9ca7bd46
0676221c
3ff0e1eb
f40fc55d
d86a3a91
1b05454b
446cfbee
e1a027f8
fa555397
765d0a24
2b8bb150
b42e8af4
e51592aa
f7a23a9d
81fda8e7
582d4e2c
2f1dd097
d8f63c95
2